<a href="https://colab.research.google.com/github/CristValen/Acciones-RNR/blob/main/nps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd

# Establecer la semilla del generador de números aleatorios
np.random.seed(42)

# Supongamos que 'df' es tu DataFrame y que 'clase' es tu variable objetivo
df = pd.read_csv('tu_archivo.csv')

# Crear las variables objetivo y características
X = df.drop('clase', axis=1)
y = df['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Crear el modelo de Random Forest
modelo = RandomForestClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda aleatoria
param_dist = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Realizar la búsqueda aleatoria de hiperparámetros
busqueda_aleatoria = RandomizedSearchCV(estimator=modelo,
                                        param_distributions=param_dist,
                                        n_iter=100,
                                        cv=3,
                                        verbose=2,
                                        random_state=42,
                                        n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_aleatoria.fit(X_train_res, y_train_res)

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas = busqueda_aleatoria.predict_proba(X_test)

# Definir tus umbrales personalizados aquí. Por ejemplo:
umbrales = [0.4, 0.3, 0.3]  # Un umbral para cada clase

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
predicciones = np.array([busqueda_aleatoria.classes_[i] for i in np.argmax(probas >= umbrales, axis=1)])

# Imprimir un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, predicciones))

# Calcular e imprimir la matriz de confusión con los nombres de las clases
cm = confusion_matrix(y_test, predicciones)
cm_df = pd.DataFrame(cm,
                     index=[i for i in busqueda_aleatoria.classes_],
                     columns=[i for i in busqueda_aleatoria.classes_])
print("Matriz de confusión:")
print(cm_df)

# Imprimir la precisión del modelo
print("Precisión del modelo:")
print(accuracy_score(y_test, predicciones))

# Calcular e imprimir la puntuación AUC-ROC para cada clase
lb = LabelBinarizer()
lb.fit(y_test)
y_test_lb = lb.transform(y_test)
predicciones_lb = lb.transform(predicciones)
print("Puntuación AUC-ROC para cada clase:")
for i in range(len(lb.classes_)):
    print(f"{lb.classes_[i]}: {roc_auc_score(y_test_lb[:, i], predicciones_lb[:, i])}")

In [ ]:
#########nuevos

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import LabelBinarizer
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd

# Establecer la semilla del generador de números aleatorios
np.random.seed(42)

# Supongamos que 'df' es tu DataFrame y que 'clase' es tu variable objetivo
df = pd.read_csv('tu_archivo.csv')

# Crear las variables objetivo y características
X = df.drop('clase', axis=1)
y = df['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Crear el modelo de Random Forest
modelo = RandomForestClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda aleatoria
param_dist = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Realizar la búsqueda aleatoria de hiperparámetros
busqueda_aleatoria = RandomizedSearchCV(estimator=modelo,
                                        param_distributions=param_dist,
                                        n_iter=100,
                                        cv=3,
                                        verbose=2,
                                        random_state=42,
                                        n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_aleatoria.fit(X_train_res, y_train_res)

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas = busqueda_aleatoria.predict_proba(X_test)

# Imprimir un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, predicciones))

# Calcular e imprimir la matriz de confusión con los nombres de las clases
cm = confusion_matrix(y_test, predicciones)
cm_df = pd.DataFrame(cm,
                     index=[i for i in busqueda_aleatoria.classes_],
                     columns=[i for i in busqueda_aleatoria.classes_])
print("Matriz de confusión:")
print(cm_df)

# Imprimir la precisión del modelo
print("Precisión del modelo:")
print(accuracy_score(y_test, predicciones))

# Calcular e imprimir la puntuación AUC-ROC para cada clase
lb = LabelBinarizer()
lb.fit(y_test)
y_test_lb = lb.transform(y_test)
predicciones_lb = lb.transform(predicciones)
print("Puntuación AUC-ROC para cada clase:")
for i in range(len(lb.classes_)):
    print(f"{lb.classes_[i]}: {roc_auc_score(y_test_lb[:, i], predicciones_lb[:, i])}")

# Función para calcular el valor KS
def calc_ks(data):
    data['bucket']=(data['probability'].rank(pct=True)*10).astype(int)
    grouped=data.groupby('bucket',as_index=True)
    kstable=grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob']=grouped.max().probability
    kstable['bads']=grouped.sum().bad
    kstable['goods']=grouped.sum().good
    kstable=kstable.reset_index()
    kstable['bad_rate']=kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks']=(kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value=kstable.ks.abs().max()
    return ks_value

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Calcular e imprimir el valor KS y el coeficiente Gini para cada clase
print("Valor KS y coeficiente Gini para cada clase:")
for i in range(len(lb.classes_)):
    data = pd.DataFrame({'label': y_test_lb[:, i], 'probability': probas[:, i]})
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    ks = calc_ks(data)
    gini = calc_gini(y_test_lb[:, i], probas[:, i])
    print(f"{lb.classes_[i]}: KS = {ks}, Gini = {gini}")



In [ ]:
###definitivo

# Establecer la semilla del generador de números aleatorios
np.random.seed(42)

# Supongamos que 'df' es tu DataFrame y que 'clase' es tu variable objetivo
df = pd.read_csv('tu_archivo.csv')

# Crear las variables objetivo y características
X = df.drop('clase', axis=1)
y = df['clase']

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Crear el modelo de Random Forest
modelo = RandomForestClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda aleatoria
param_dist = {
    'n_estimators': [50, 100, 200, 300, 400],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Realizar la búsqueda aleatoria de hiperparámetros
busqueda_aleatoria = RandomizedSearchCV(estimator=modelo,
                                        param_distributions=param_dist,
                                        n_iter=100,
                                        cv=3,
                                        verbose=2,
                                        random_state=42,
                                        n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_aleatoria.fit(X_train_res, y_train_res)

# Obtener las probabilidades de las predicciones en el conjunto de validación y prueba
probas_val = busqueda_aleatoria.predict_proba(X_val)
probas_test = busqueda_aleatoria.predict_proba(X_test)

# Imprimir un informe de clasificación para validación y prueba
print("Informe de clasificación para validación:")
print(classification_report(y_val, probas_val))
print("Informe de clasificación para prueba:")
print(classification_report(y_test, probas_test))

# Calcular e imprimir la matriz de confusión con los nombres de las clases para validación y prueba
cm_val = confusion_matrix(y_val, probas_val)
cm_test = confusion_matrix(y_test, probas_test)
cm_df_val = pd.DataFrame(cm_val,
                     index=[i for i in busqueda_aleatoria.classes_],
                     columns=[i for i in busqueda_aleatoria.classes_])
cm_df_test = pd.DataFrame(cm_test,
                     index=[i for i in busqueda_aleatoria.classes_],
                     columns=[i for i in busqueda_aleatoria.classes_])
print("Matriz de confusión para validación:")
print(cm_df_val)
print("Matriz de confusión para prueba:")
print(cm_df_test)

# Imprimir la precisión del modelo para validación y prueba
print("Precisión del modelo para validación:")
print(accuracy_score(y_val, probas_val))
print("Precisión del modelo para prueba:")
print(accuracy_score(y_test, probas_test))

# Calcular e imprimir la puntuación AUC-ROC para cada clase para validación y prueba
lb = LabelBinarizer()
lb.fit(y_val)
y_val_lb = lb.transform(y_val)
probas_val_lb = lb.transform(probas_val)
y_test_lb = lb.transform(y_test)
probas_test_lb = lb.transform(probas_test)
print("Puntuación AUC-ROC para cada clase para validación:")
for i in range(len(lb.classes_)):
    print(f"{lb.classes_[i]}: {roc_auc_score(y_val_lb[:, i], probas_val_lb[:, i])}")
print("Puntuación AUC-ROC para cada clase para prueba:")
for i in range(len(lb.classes_)):
    print(f"{lb.classes_[i]}: {roc_auc_score(y_test_lb[:, i], probas_test_lb[:, i])}")

# Función para calcular el valor KS
def calc_ks(data):
    data['bucket']=(data['probability'].rank(pct=True)*10).astype(int)
    grouped=data.groupby('bucket',as_index=True)
    kstable=grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob']=grouped.max().probability
    kstable['bads']=grouped.sum().bad
    kstable['goods']=grouped.sum().good
    kstable=kstable.reset_index()
    kstable['bad_rate']=kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks']=(kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value=kstable.ks.abs().max()
    return ks_value

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Calcular e imprimir el valor KS y el coeficiente Gini para cada clase para validación y prueba
print("Valor KS y coeficiente Gini para cada clase para validación:")
for i in range(len(lb.classes_)):
    data = pd.DataFrame({'label': y_val_lb[:, i], 'probability': probas_val[:, i]})
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    ks = calc_ks(data)
    gini = calc_gini(y_val_lb[:, i], probas_val[:, i])
    print(f"{lb.classes_[i]}: KS = {ks}, Gini = {gini}")
print("Valor KS y coeficiente Gini para cada clase para prueba:")
for i in range(len(lb.classes_)):
    data = pd.DataFrame({'label': y_test_lb[:, i], 'probability': probas_test[:, i]})
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    ks = calc_ks(data)
    gini = calc_gini(y_test_lb[:, i], probas_test[:, i])
    print(f"{lb.classes_[i]}: KS = {ks}, Gini = {gini}")


In [ ]:
from pyspark.sql.functions import to_date

# Asumiendo que tu DataFrame se llama df y tus columnas son 'ope' y 'conta'
df = df.withColumn("ope", to_date(df['ope'], "yyyy-MM-dd"))
df = df.withColumn("conta", to_date(df['conta'], "yyyy-MM-dd"))

# Agrupar por la columna 'ope'
df_grouped_by_ope = df.groupBy("ope").count()

# Agrupar por la columna 'conta'
df_grouped_by_conta = df.groupBy("conta").count()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pandas as pd

# Establecer la semilla del generador de números aleatorios
np.random.seed(42)

# Supongamos que 'df' es tu DataFrame y que 'clase' es tu variable objetivo
df = pd.read_csv('tu_archivo.csv')

# Crear las variables objetivo y características
X = df.drop('clase', axis=1)
y = df['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear una instancia del clasificador Gradient Boosting
modelo = GradientBoostingClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda en grilla
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
}

# Realizar la búsqueda en grilla de hiperparámetros
busqueda_en_grilla = GridSearchCV(estimator=modelo,
                                  param_grid=param_grid,
                                  cv=3,
                                  verbose=2,
                                  n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_en_grilla.fit(X_train, y_train)

# Predecir las clases para el conjunto de prueba
y_pred = busqueda_en_grilla.predict(X_test)

# Definir tus umbrales personalizados aquí. Por ejemplo:
umbrales = [0.4, 0.3, 0.3]  # Un umbral para cada clase

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
probas = busqueda_en_grilla.predict_proba(X_test)
predicciones = np.array([busqueda_en_grilla.classes_[i] for i in np.argmax(probas >= umbrales, axis=1)])

# Imprimir un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, predicciones))

# Calcular e imprimir la matriz de confusión con los nombres de las clases
cm = confusion_matrix(y_test, predicciones)
cm_df = pd.DataFrame(cm,
                     index=[i for i in busqueda_en_grilla.classes_],
                     columns=[i for i in busqueda_en_grilla.classes_])
print("Matriz de confusión:")
print(cm_df)

# Imprimir la precisión del modelo
print("Precisión del modelo:")
print(accuracy_score(y_test, predicciones))

# Calcular e imprimir la puntuación AUC-ROC para cada clase
lb = LabelBinarizer()
lb.fit(y_test)
y_test_lb = lb.transform(y_test)
predicciones_lb = lb.transform(predicciones)
print("Puntuación AUC-ROC para cada clase:")
for i in range(len(lb.classes_)):
    print(f"{lb.classes_[i]}: {roc_auc_score(y_test_lb[:, i], predicciones_lb[:, i])}")


In [ ]:


import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, label_binarize

# Establecer la semilla para la reproducibilidad
np.random.seed(0)

# Supongamos que X es tu conjunto de datos e y son las etiquetas correspondientes

# Crear el codificador
le = LabelEncoder()

# Ajustar el codificador y transformar las etiquetas
y = le.fit_transform(y)

# Convertir los datos a float32
X = X.astype(np.float32)
y = y.astype(np.float32)

# División de los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Creación del modelo MLP más profundo
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Tres neuronas en la capa de salida para los tres niveles de la variable dependiente

# Compilación del modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Asignación de pesos a las clases (supongamos que class_weights es un diccionario que contiene los pesos)
class_weights = {0: 1., 1: 1., 2: 1.}  # Modifica esto según tus necesidades

# Entrenamiento del modelo
model.fit(X_train, y_train, epochs=100, verbose=0, class_weight=class_weights)

# Predicción en el conjunto de prueba
y_pred_proba = model.predict(X_test)

# Definir los umbrales de decisión para cada clase
thresholds = [0.5, 0.5, 0.5]  # Modifica esto según tus necesidades

y_pred = np.zeros_like(y_pred_proba)
for i in range(3):
    y_pred[:, i] = (y_pred_proba[:, i] > thresholds[i]).astype(int)

y_pred = np.argmax(y_pred, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular la matriz de confusión para todas las categorías
confusion = confusion_matrix(y_test, y_pred)

# Crear un DataFrame para visualizar la matriz de confusión con los nombres de las clases
class_names = ['detractor', 'neutro', 'promotor']  # Modifica esto según tus necesidades
confusion_df = pd.DataFrame(confusion, index=class_names, columns=class_names)

print("Confusion Matrix: \n", confusion_df)

# Calcular las métricas para cada categoría
for i in range(3):
    precision = precision_score(y_test == i, y_pred == i)
    recall = recall_score(y_test == i, y_pred == i)
    f1 = f1_score(y_test == i, y_pred == i)

    # Calcular la curva ROC y el AUC-ROC
    fpr, tpr, _ = roc_curve(y_test == i, y_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)

    print(f"Clase {class_names[i]}:")
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1)
    print("ROC AUC Score: ", roc_auc)


In [ ]:
# Calcular las métricas ponderadas para cada categoría
for i in range(3):
    precision = precision_score(y_test == i, y_pred == i, average='weighted')
    recall = recall_score(y_test == i, y_pred == i, average='weighted')
    f1 = f1_score(y_test == i, y_pred == i, average='weighted')

    print(f"Clase {class_names[i]} ponderada:")
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1)

### a mano
from sklearn.metrics import f1_score

# Supongamos que 'y_test' son tus etiquetas verdaderas e 'y_pred' son tus predicciones
f1_scores = [f1_score(y_test == i, y_pred == i) for i in range(3)]
class_counts = [sum(y_test == i) for i in range(3)]
total_count = len(y_test)

weighted_f1_score = sum(f1 * count for f1, count in zip(f1_scores, class_counts)) / total_count

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve, auc, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pandas as pd

# Establecer la semilla del generador de números aleatorios
np.random.seed(42)

# Supongamos que 'df' es tu DataFrame y que 'clase' es tu variable objetivo
df = pd.read_csv('tu_archivo.csv')

# Crear las variables objetivo y características
X = df.drop('clase', axis=1)
y = df['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear una instancia del clasificador Gradient Boosting
modelo = GradientBoostingClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda en grilla
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
}

# Realizar la búsqueda en grilla de hiperparámetros
busqueda_en_grilla = GridSearchCV(estimator=modelo,
                                  param_grid=param_grid,
                                  cv=3,
                                  verbose=2,
                                  n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_en_grilla.fit(X_train, y_train)

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas = busqueda_en_grilla.predict_proba(X_test)

# Definir tus umbrales personalizados aquí. Por ejemplo:
umbrales = [0.4, 0.3, 0.3]  # Un umbral para cada clase

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
predicciones = np.array([busqueda_en_grilla.classes_[i] for i in np.argmax(probas >= umbrales, axis=1)])

# Imprimir un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, predicciones))

# Calcular e imprimir la matriz de confusión con los nombres de las clases
cm = confusion_matrix(y_test, predicciones)
cm_df = pd.DataFrame(cm,
                     index=[i for i in busqueda_en_grilla.classes_],
                     columns=[i for i in busqueda_en_grilla.classes_])
print("Matriz de confusión:")
print(cm_df)

# Imprimir la precisión del modelo
print("Precisión del modelo:")
print(accuracy_score(y_test, predicciones))

# Calcular e imprimir la puntuación AUC-ROC para cada clase
lb = LabelBinarizer()
lb.fit(y_test)
y_test_lb = lb.transform(y_test)
predicciones_lb = lb.transform(predicciones)
print("Puntuación AUC-ROC para cada clase:")
for i in range(len(lb.classes_)):
    print(f"{lb.classes_[i]}: {roc_auc_score(y_test_lb[:, i], predicciones_lb[:, i])}")

# Función para calcular el valor KS
def calc_ks(data):
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    data['bucket']=(data['probability'].rank(pct=True)*10).astype(int)
    grouped=data.groupby('bucket',as_index=True)
    kstable=grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob']=grouped.max().probability
    kstable['bads']=grouped.sum().bad
    kstable['goods']=grouped.sum().good
    kstable=kstable.reset_index()
    kstable['bad_rate']=kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks']=(kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value=kstable.ks.abs().max()
    return ks_value

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Calcular e imprimir el valor KS y el coeficiente Gini para cada clase
print("Valor KS y coeficiente Gini para cada clase:")
for i in range(len(lb.classes_)):
    data = pd.DataFrame({'label': y_test_lb[:, i], 'probability': probas[:, i]})
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    ks = calc_ks(data)
    gini = calc_gini(y_test_lb[:, i], probas[:, i])
    print(f"{lb.classes_[i]}: KS = {ks}, Gini = {gini}")



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import LabelEncoder, label_binarize

# Establecer la semilla para la reproducibilidad
np.random.seed(0)

# Supongamos que X2 es tu conjunto de datos e y2 son las etiquetas correspondientes
X2 = X.copy()
y2 = y.copy()

# Crear el codificador
le2 = LabelEncoder()

# Ajustar el codificador y transformar las etiquetas
y2 = le2.fit_transform(y2)

# Convertir los datos a float32
X2 = X2.astype(np.float32)
y2 = y2.astype(np.float32)

# División de los datos en conjuntos de entrenamiento y prueba
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=0)

# Creación del modelo MLP más profundo
model2 = Sequential()
model2.add(Dense(50, activation='relu', input_shape=(X_train.shape[1],)))
model2.add(Dense(50, activation='relu'))
model2.add(Dense(50, activation='relu'))
model2.add(Dense(3, activation='softmax'))  # Tres neuronas en la capa de salida para los tres niveles de la variable dependiente

# Compilación del modelo
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Asignación de pesos a las clases (supongamos que class_weights es un diccionario que contiene los pesos)
class_weights = {0: 1., 1: 1., 2: 1.}  # Modifica esto según tus necesidades

# Entrenamiento del modelo
model2.fit(X_train2, y_train2, epochs=100, verbose=0, class_weight=class_weights)

# Predicción en el conjunto de prueba
y_pred_proba2 = model.predict(X_test)

# Definir los umbrales de decisión para cada clase
thresholds = [0.5, 0.5, 0.5]  # Modifica esto según tus necesidades

y_pred2 = np.zeros_like(y_pred_proba)
for i in range(3):
    y_pred[:, i] = (y_pred_proba[:, i] > thresholds[i]).astype(int)

y_pred = np.argmax(y_pred, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular la matriz de confusión para todas las categorías
confusion = confusion_matrix(y_test, y_pred)

# Crear un DataFrame para visualizar la matriz de confusión con los nombres de las clases
class_names = ['detractor', 'neutro', 'promotor']  # Modifica esto según tus necesidades
confusion_df = pd.DataFrame(confusion, index=class_names, columns=class_names)

print("Confusion Matrix: \n", confusion_df)

# Calcular las métricas para cada categoría
for i in range(3):
    precision = precision_score(y_test == i, y_pred == i)
    recall = recall_score(y_test == i, y_pred == i)
    f1 = f1_score(y_test == i, y_pred == i)

    # Calcular la curva ROC y el AUC-ROC
    fpr, tpr, _ = roc_curve(y_test == i, y_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)

    print(f"Clase {class_names[i]}:")
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1)
    print("ROC AUC Score: ", roc_auc)

# Función para calcular el valor KS
def calc_ks(data):
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    data['bucket']=(data['probability'].rank(pct=True)*10).astype(int)
    grouped=data.groupby('bucket',as_index=True)
    kstable=grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob']=grouped.max().probability
    kstable['bads']=grouped.sum().bad
    kstable['goods']=grouped.sum().good
    kstable=kstable.reset_index()
    kstable['bad_rate']=kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks']=(kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value=kstable.ks.abs().max()
    return ks_value

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Calcular e imprimir el valor KS y el coeficiente Gini para cada clase
print("Valor KS y coeficiente Gini para cada clase:")
for i in range(len(class_names)):
    data = pd.DataFrame({'label': (y_test == i).astype(int), 'probability': y_pred_proba[:, i]})
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    ks = calc_ks(data)
    gini = calc_gini((y_test == i).astype(int), y_pred_proba[:, i])
    print(f"{class_names[i]}: KS = {ks}, Gini = {gini}")


    from scipy.optimize import differential_evolution

# Definir la función objetivo para la optimización
def objective(thresholds):
    # Aplicar los umbrales a las probabilidades para obtener las predicciones finales
    y_pred = np.zeros_like(y_pred_proba2)
    for i in range(3):
        y_pred[:, i] = (y_pred_proba2[:, i] > thresholds[i]).astype(int)
    y_pred = np.argmax(y_pred, axis=1)

    # Calcular las métricas
    roc_auc_scores = []
    ks_values = []
    gini_values = []
    for i in range(len(class_names)):
        data = pd.DataFrame({'label': (y_test2 == i).astype(int), 'probability': y_pred_proba2[:, i]})
        data['good']=(data['label']==0).astype(int)
        data['bad']=(data['label']==1).astype(int)
        ks = calc_ks(data)
        gini = calc_gini((y_test2 == i).astype(int), y_pred_proba2[:, i])
        fpr, tpr, _ = roc_curve((y_test2 == i).astype(int), y_pred_proba2[:, i])
        roc_auc = auc(fpr, tpr)

        roc_auc_scores.append(roc_auc)
        ks_values.append(ks)
        gini_values.append(gini)

    # Calcular el promedio de las métricas
    avg_roc_auc = np.mean(roc_auc_scores)
    avg_ks = np.mean(ks_values)
    avg_gini = np.mean(gini_values)

    # La función objetivo es el negativo del promedio de las métricas porque queremos maximizarlas
    return -(avg_roc_auc + avg_ks + avg_gini) / 3

# Definir los límites para los umbrales
bounds = [(0, 1)] * 3

# Realizar la optimización
result = differential_evolution(objective, bounds)

# Imprimir los mejores umbrales encontrados
print("Mejores umbrales: ", result.x)



from scipy.optimize import differential_evolution

# Definir la función objetivo para la optimización
def objective(weights):
    # Asignar los pesos a las clases
    class_weights = {i: weights[i] for i in range(len(weights))}

    # Entrenar el modelo con los pesos de las clases
    model2.fit(X_train2, y_train2, epochs=100, verbose=0, class_weight=class_weights)

    # Predicción en el conjunto de prueba
    y_pred_proba2 = model.predict(X_test)

    # Aplicar los umbrales a las probabilidades para obtener las predicciones finales
    y_pred = np.zeros_like(y_pred_proba)
    for i in range(3):
        y_pred[:, i] = (y_pred_proba[:, i] > thresholds[i]).astype(int)
    y_pred = np.argmax(y_pred, axis=1)

    # Calcular las métricas
    roc_auc_scores = []
    ks_values = []
    gini_values = []
    for i in range(len(class_names)):
        data = pd.DataFrame({'label': (y_test2 == i).astype(int), 'probability': y_pred_proba2[:, i]})
        data['good']=(data['label']==0).astype(int)
        data['bad']=(data['label']==1).astype(int)
        ks = calc_ks(data)
        gini = calc_gini((y_test2 == i).astype(int), y_pred_proba2[:, i])
        fpr, tpr, _ = roc_curve((y_test2 == i).astype(int), y_pred_proba2[:, i])
        roc_auc = auc(fpr, tpr)

        roc_auc_scores.append(roc_auc)
        ks_values.append(ks)
        gini_values.append(gini)

    # Calcular el promedio de las métricas
    avg_roc_auc = np.mean(roc_auc_scores)
    avg_ks = np.mean(ks_values)
    avg_gini = np.mean(gini_values)

    # La función objetivo es el negativo del promedio de las métricas porque queremos maximizarlas
    return -(avg_roc_auc + avg_ks + avg_gini) / 3

# Definir los límites para los pesos (supongamos que los pesos están entre 0 y 10)
bounds = [(0, 10)] * 3

# Realizar la optimización
result = differential_evolution(objective, bounds)

# Imprimir los mejores pesos encontrados
print("Mejores pesos: ", result.x)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import LabelBinarizer
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd

# Establecer la semilla del generador de números aleatorios
np.random.seed(42)

# Supongamos que 'df' es tu DataFrame y que 'clase' es tu variable objetivo
df = pd.read_csv('tu_archivo.csv')

# Crear las variables objetivo y características
X = df.drop('clase', axis=1)
y = df['clase']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Crear el modelo SVM
modelo = SVC(probability=True, random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda aleatoria
param_dist = {
    'C': [0.1, 1, 10],
    'gamma': [1, 0.1, 0.01],
    'kernel': ['rbf', 'poly']
}

# Realizar la búsqueda aleatoria de hiperparámetros
busqueda_aleatoria = RandomizedSearchCV(estimator=modelo,
                                        param_distributions=param_dist,
                                        n_iter=10,
                                        cv=3,
                                        verbose=2,
                                        random_state=42,
                                        n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_aleatoria.fit(X_train_res, y_train_res)

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas = busqueda_aleatoria.predict_proba(X_test)

# Definir tus umbrales personalizados aquí. Por ejemplo:
umbrales = [0.4, 0.3, 0.3]  # Un umbral para cada clase

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
predicciones = np.array([busqueda_aleatoria.classes_[i] for i in np.argmax(probas >= umbrales, axis=1)])

# Imprimir un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, predicciones))

# Calcular e imprimir la matriz de confusión con los nombres de las clases
cm = confusion_matrix(y_test, predicciones)
cm_df = pd.DataFrame(cm,
                     index=[i for i in busqueda_aleatoria.classes_],
                     columns=[i for i in busqueda_aleatoria.classes_])
print("Matriz de confusión:")
print(cm_df)

# Imprimir la precisión del modelo
print("Precisión del modelo:")
print(accuracy_score(y_test, predicciones))

# Calcular e imprimir la puntuación AUC-ROC para cada clase
lb = LabelBinarizer()
lb.fit(y_test)
y_test_lb = lb.transform(y_test)
predicciones_lb = lb.transform(predicciones)
print("Puntuación AUC-ROC para cada clase:")
for i in range(len(lb.classes_)):
    print(f"{lb.classes_[i]}: {roc_auc_score(y_test_lb[:, i], predicciones_lb[:, i])}")

# Función para calcular el valor KS
def calc_ks(data):
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    data['bucket']=(data['probability'].rank(pct=True)*10).astype(int)
    grouped=data.groupby('bucket',as_index=True)
    kstable=grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob']=grouped.max().probability
    kstable['bads']=grouped.sum().bad
    kstable['goods']=grouped.sum().good
    kstable=kstable.reset_index()
    kstable['bad_rate']=kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks']=(kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value=kstable.ks.abs().max()
    return ks_value

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Calcular e imprimir el valor KS y el coeficiente Gini para cada clase
print("Valor KS y coeficiente Gini para cada clase:")
for i in range(len(lb.classes_)):
    data = pd.DataFrame({'label': (y_test == i).astype(int), 'probability': probas[:, i]})
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    ks = calc_ks(data)
    gini = calc_gini((y_test == i).astype(int), probas[:, i])
    print(f"{lb.classes_[i]}: KS = {ks}, Gini = {gini}")

In [ ]:

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RelacionarTablas").getOrCreate()

# Carga las tablas
Pedidos = spark.read.table("Pedidos")
DetallePedidos = spark.read.table("DetallePedidos")
Productos = spark.read.table("Productos")
CodSubCategoria = spark.read.table("CodSubCategoria")
CodCategoria = spark.read.table("CodCategoria")
Clientes = spark.read.table("Clientes")
Ciudades = spark.read.table("Ciudades")
Paises = spark.read.table("Paises")
ModoEnvio = spark.read.table("ModoEnvio")
Segmento = spark.read.table("Segmento")

# Relaciona las tablas
df1 = Pedidos.join(DetallePedidos, Pedidos.NroPedido == DetallePedidos.NroPedido)
df2 = df1.join(Productos, df1.IDProducto == Productos.NroProducto)
df3 = df2.join(CodSubCategoria, df2.CodSubCategoria == CodSubCategoria.CODSubCategoria)
df4 = df3.join(CodCategoria, df3.CODCategoria == CodCategoria.CODCategoria)
df5 = df4.join(Clientes, df4.IDCliente == Clientes.NroCliente)
df6 = df5.join(Ciudades, df5.CiudadDespacho == Ciudades.CODCiudad)
df7 = df6.join(Paises, Ciudades.Pais == Paises.CodigoPais)
df8 = df7.join(ModoEnvio, df7.IDModoEnvio == ModoEnvio.CODModoEnvio)
final_df = df8.join(Segmento, Clientes.IDSegmento == Segmento.CODSegmento)

# Muestra el resultado
final_df.show()

In [ ]:

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# Asegúrate de que 'Marfen' y 'Precio' sean de tipo numérico
final_df = final_df.withColumn("Marfen", final_df["Marfen"].cast("double"))
final_df = final_df.withColumn("Precio", final_df["Precio"].cast("double"))

# Crea un VectorAssembler que combina las columnas de características en una sola columna de vector
vectorAssembler = VectorAssembler(inputCols=["Marfen"], outputCol="features")

# Transforma los datos
df = vectorAssembler.transform(final_df)

# Crea el modelo de regresión lineal
lr = LinearRegression(featuresCol="features", labelCol="Precio")

# Ajusta el modelo a los datos
lr_model = lr.fit(df)

# Imprime los coeficientes y la intersección del modelo
print("Coeficientes: " + str(lr_model.coefficients))
print("Intersección: " + str(lr_model.intercept))

In [ ]:

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Supongamos que X son tus datos
# X = ...

# Calcula las sumas de los cuadrados dentro del cluster para diferentes valores de k
sse = []
silhouette = []
list_k = list(range(2, 10))

for k in list_k:
    km = KMeans(n_clusters=k)
    km.fit(X)

    # suma de los cuadrados dentro del cluster
    sse.append(km.inertia_)

    # coeficiente de silueta
    silhouette.append(silhouette_score(X, km.labels_))

# Grafica sse en función de k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Número de clusters *k*')
plt.ylabel('Suma de los cuadrados dentro del cluster')
plt.show()

# Grafica el coeficiente de silueta en función de k
plt.figure(figsize=(6, 6))
plt.plot(list_k, silhouette, '-o')
plt.xlabel(r'Número de clusters *k*')
plt.ylabel('Coeficiente de Silueta')
plt.show()

In [ ]:

import pandas as pd
from scipy import stats

# Supongamos que df es tu DataFrame y las columnas son 'ModoEnvio', 'FechaPedido' y 'FechaDespacho'
# df = ...

# Calcula la diferencia en días entre la fecha de pedido y la fecha de despacho
df['Dias'] = (df['FechaDespacho'] - df['FechaPedido']).dt.days

# Obtiene los modos de envío únicos
modos_envio = df['ModoEnvio'].unique()

# Realiza un test t para cada par de modos de envío
for i in range(len(modos_envio)):
    for j in range(i+1, len(modos_envio)):
        modo_envio1 = df[df['ModoEnvio'] == modos_envio[i]]['Dias']
        modo_envio2 = df[df['ModoEnvio'] == modos_envio[j]]['Dias']

        t_stat, p_val = stats.ttest_ind(modo_envio1, modo_envio2)

        print(f"Modo de envío: {modos_envio[i]} vs {modos_envio[j]}")
        print(f"Estadística t: {t_stat}")
        print(f"Valor p: {p_val}\n")

In [ ]:

from pyspark.sql.functions import col

# Supongamos que df es tu DataFrame
# df = ...

# Obtiene la lista de columnas categóricas
categoricas = [col.name for col in df.schema.fields if col.dataType == StringType()]

print("Variables categóricas: ", categoricas)

In [ ]:

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RelacionarTablas").getOrCreate()

# Carga las tablas
Pedidos = spark.read.table("Pedidos").withColumnRenamed("NroPedido", "NumeroPedido")
DetallePedidos = spark.read.table("DetallePedidos").withColumnRenamed("NroPedido", "NumeroPedido")
Productos = spark.read.table("Productos").withColumnRenamed("CodSubCategoria", "CodSubCategoriaProductos")
CodSubCategoria = spark.read.table("CodSubCategoria").withColumnRenamed("CODCategoria", "CODCategoriaSub")
CodCategoria = spark.read.table("CodCategoria")
Clientes = spark.read.table("Clientes")
Ciudades = spark.read.table("Ciudades")
Paises = spark.read.table("Paises")
ModoEnvio = spark.read.table("ModoEnvio")
Segmento = spark.read.table("Segmento")

# Relaciona las tablas
df1 = Pedidos.join(DetallePedidos, Pedidos.NumeroPedido == DetallePedidos.NumeroPedido)
df2 = df1.join(Productos, df1.IDProducto == Productos.NroProducto)
df3 = df2.join(CodSubCategoria, df2.CodSubCategoriaProductos == CodSubCategoria.CODSubCategoria)
df4 = df3.join(CodCategoria, df3.CODCategoriaSub == CodCategoria.CODCategoria)
df5 = df4.join(Clientes, df4.IDCliente == Clientes.NroCliente)
df6 = df5.join(Ciudades, df5.CiudadDespacho == Ciudades.CODCiudad)
df7 = df6.join(Paises, Ciudades.Pais == Paises.CodigoPais)
df8 = df7.join(ModoEnvio, df7.IDModoEnvio == ModoEnvio.CODModoEnvio)
final_df = df8.join(Segmento, Clientes.IDSegmento == Segmento.CODSegmento)

# Muestra el resultado
final_df.show()

In [ ]:

from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Supongamos que df es tu DataFrame y categoricas es una lista de las columnas categóricas
# df = ...
# categoricas = ...

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in categoricas]
encoder = OneHotEncoder(inputCols=[indexer.getOutputCol() for indexer in indexers], outputCols=["{0}_encoded".format(indexer.getOutputCol()) for indexer in indexers])

pipeline = Pipeline(stages=indexers + [encoder])
df = pipeline.fit(df).transform(df)

In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import lit

# Lista de columnas categóricas
cat_cols = [item[0] for item in final_df.dtypes if item[1].startswith('string')]

indexers = []

for column in cat_cols:
    if column + "_index" not in final_df.columns:
        indexer = StringIndexer(inputCol=column, outputCol=column+"_index").fit(final_df)
        indexers.append(indexer)

pipeline = Pipeline(stages=indexers)
final_df = pipeline.fit(final_df).transform(final_df)

# Lista de columnas para el VectorAssembler
input_cols = [column+"_index" for column in cat_cols]

if "features" not in final_df.columns:
    vec_assembler = VectorAssembler(inputCols=input_cols, outputCol="features")
    final_df = vec_assembler.transform(final_df)

# Aplicar PCA
pca = PCA(k=min(final_df.select("features").count(), 27), inputCol="features", outputCol="pca_features")
model = pca.fit(final_df)
result = model.transform(final_df)

# Encontrar el número de componentes necesarios para explicar el 95% de la varianza
variances = model.explainedVariance
cum_sum = variances.cumsum()
n_components = len(cum_sum[cum_sum < 0.95]) + 1

print(f"Se necesitan {n_components} componentes para explicar el 95% de la varianza.")




In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
import matplotlib.pyplot as plt

# Lista de columnas categóricas
cat_cols = [item[0] for item in final_df.dtypes if item[1].startswith('string')]

indexers = []

for column in cat_cols:
    if column + "_index" not in final_df.columns:
        indexer = StringIndexer(inputCol=column, outputCol=column+"_index").fit(final_df)
        indexers.append(indexer)

pipeline = Pipeline(stages=indexers)
final_df = pipeline.fit(final_df).transform(final_df)

# Lista de columnas para el VectorAssembler
input_cols = [column+"_index" for column in cat_cols]

if "features" not in final_df.columns:
    vec_assembler = VectorAssembler(inputCols=input_cols, outputCol="features")
    final_df = vec_assembler.transform(final_df)

# Aplicar PCA
if "pca_features" not in final_df.columns:
    pca = PCA(k=min(final_df.select("features").count(), 27), inputCol="features", outputCol="pca_features")
    model = pca.fit(final_df)
    final_df = model.transform(final_df)

# Encontrar el número de componentes necesarios para explicar el 95% de la varianza
variances = model.explainedVariance
cum_sum = variances.cumsum()
n_components = len(cum_sum[cum_sum < 0.95]) + 1

print(f"Se necesitan {n_components} componentes para explicar el 95% de la varianza.")

# Aplicar KMeans a los datos reducidos
if "prediction" not in final_df.columns:
    kmeans = KMeans(featuresCol='pca_features', predictionCol="prediction", k=3)  # Ajusta 'k' según tus necesidades
    kmeans_model = kmeans.fit(final_df)
    final_df = kmeans_model.transform(final_df)

# Convertir la columna 'prediction' a float
to_float = udf(lambda x: float(x), FloatType())
final_df = final_df.withColumn("cluster", to_float(final_df.prediction))

# Seleccionar las primeras dos componentes principales para visualizar
pandas_df = final_df.select("pca_features").rdd.map(lambda x: (float(x[0][0]), float(x[0][1]))).toDF(["PCA1", "PCA2"]).toPandas()
pandas_df['cluster'] = final_df.select("cluster").toPandas()

plt.scatter(pandas_df['PCA1'], pandas_df['PCA2'], c=pandas_df['cluster'])
plt.show()





In [ ]:
# Importar las funciones necesarias de PySpark
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

# Definir la UDF para calcular el valor total de un pedido
def calcular_total(precio, descuento, margen):
    precio_descuento = precio - (precio * descuento / 100)
    total = precio_descuento + (precio_descuento * margen / 100)
    return total

calcular_total_udf = udf(calcular_total, FloatType())

# Aplicar la UDF al DataFrame
final_df = final_df.withColumn("total_pedido", calcular_total_udf(final_df['precio'], final_df['descuento'], final_df['margen']))

# Definir la UDF para encontrar el pedido con el mayor total
def maximo_pedido(total):
    return max(total)

maximo_pedido_udf = udf(maximo_pedido, FloatType())

# Aplicar la UDF al DataFrame
final_df = final_df.withColumn("maximo_pedido", maximo_pedido_udf(final_df['total_pedido']))



In [ ]:
# Importar las funciones necesarias de PySpark
from pyspark.sql.functions import udf, max
from pyspark.sql.types import FloatType

# Definir la UDF para calcular el valor total de un pedido
def calcular_total(precio, descuento, margen):
    precio_descuento = precio - (precio * descuento / 100)
    total = precio_descuento + (precio_descuento * margen / 100)
    return total

calcular_total_udf = udf(calcular_total, FloatType())

# Aplicar la UDF al DataFrame
final_df = final_df.withColumn("total_pedido", calcular_total_udf(final_df['precio'], final_df['descuento'], final_df['margen']))

# Encontrar el pedido con el mayor total usando funciones incorporadas de PySpark
maximo_pedido = final_df.orderBy(final_df['total_pedido'].desc()).first()

print("El pedido con el mayor total es: ", maximo_pedido)


In [ ]:
from pyspark.sql import functions as F

# Calcula el primer y tercer cuartil
cuartiles = final_df.approxQuantile('nombre_de_tu_columna', [0.25, 0.75], 0)

# Calcula el rango intercuartil
IQR = cuartiles[1] - cuartiles[0]

# Define los límites para los outliers
limite_inferior = cuartiles[0] - 1.5 * IQR
limite_superior = cuartiles[1] + 1.5 * IQR

# Filtra los outliers
outliers = final_df.filter((F.col('nombre_de_tu_columna') < limite_inferior) | (F.col('nombre_de_tu_columna') > limite_superior))

# Cuenta los outliers
numero_de_outliers = outliers.count()

print("Número de outliers: ", numero_de_outliers)


In [ ]:
from pyspark.sql import functions as F

# Calcula el promedio y la desviación estándar
stats = final_df.select(
    F.mean(F.col('nombre_de_tu_columna')).alias('media'),
    F.stddev(F.col('nombre_de_tu_columna')).alias('desviacion_estandar')
).collect()[0]

# Define los límites para los outliers
limite_inferior = stats.media - 3 * stats.desviacion_estandar
limite_superior = stats.media + 3 * stats.desviacion_estandar

# Filtra los outliers
outliers = final_df.filter((F.col('nombre_de_tu_columna') < limite_inferior) | (F.col('nombre_de_tu_columna') > limite_superior))

print("Outliers:")
outliers.show()



In [ ]:
from pyspark.sql import functions as F

# Filtra el DataFrame para incluir solo las filas con ID = 123
df_filtrado = final_df.filter(F.col('ID') == 123)

# Agrupa por la columna 'fecha' y calcula la media, mediana y desviación estándar
resultados = df_filtrado.groupBy('fecha').agg(
    F.mean(F.col('precio_unitario')).alias('media'),
    F.expr('percentile_approx(precio_unitario, 0.5)').alias('mediana'),
    F.stddev(F.col('precio_unitario')).alias('desviacion_estandar')
)

resultados.show()


Si la media y la mediana de los precios de tu producto son iguales, y la desviación estándar es 0, esto indica que todos los precios de tu producto son iguales a 28.35 a lo largo de todos los períodos. En otras palabras, no hay variabilidad en los precios de tu producto; son constantes en el tiempo. Esto podría ser el caso si el precio de tu producto no cambia o si todos los datos disponibles son del mismo período de tiempo en el que el precio fue 28.35


In [ ]:
from pyspark.sql.types import IntegerType

# Cambia el tipo de datos de la columna 'total_pedido' a entero
final_df = final_df.withColumn('total_pedido', final_df['total_pedido'].cast(IntegerType()))

# Muestra el DataFrame para verificar el cambio
final_df.show()


In [ ]:
from pyspark import SparkConf, SparkContext

# Configura Spark
conf = SparkConf().setAppName("load text file").setMaster("local[*]")
sc = SparkContext(conf=conf)

# Carga el archivo de texto
text_file = sc.textFile("harry.txt")

# Muestra las primeras líneas del archivo
for line in text_file.take(10):
    print(line)


In [ ]:
import os

# Obtén la ruta del directorio actual
dir_path = os.getcwd()

# Añade el nombre de tu archivo a la ruta del directorio
file_path = os.path.join(dir_path, 'harry.txt')

print(file_path)


In [ ]:
####### vla test boosting

# Importar las bibliotecas necesarias
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE

# Establecer la semilla del generador de números aleatorios
np.random.seed(42)

# Supongamos que 'df' es tu DataFrame y que 'clase' es tu variable objetivo
df = pd.read_csv('tu_archivo.csv')

# Crear las variables objetivo y características
X = df.drop('clase', axis=1)
y = df['clase']

# Dividir los datos en conjuntos de entrenamiento (60%) y temporal (40%)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Luego, dividir el conjunto temporal en conjuntos de prueba y validación (ambos del 20% del conjunto original)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Crear una instancia del clasificador Gradient Boosting
modelo = GradientBoostingClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda en grilla
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
}

# Realizar la búsqueda en grilla de hiperparámetros
busqueda_en_grilla = GridSearchCV(estimator=modelo,
                                  param_grid=param_grid,
                                  cv=3,
                                  verbose=2,
                                  n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_en_grilla.fit(X_train_res, y_train_res)

# Definir tus umbrales personalizados aquí. Por ejemplo:
umbrales = [0.4, 0.3, 0.3]  # Un umbral para cada clase

# Función para calcular el valor KS
def calc_ks(data):
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    data['bucket']=(data['probability'].rank(pct=True)*10).astype(int)
    grouped=data.groupby('bucket',as_index=True)
    kstable=grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob']=grouped.max().probability
    kstable['bads']=grouped.sum().bad
    kstable['goods']=grouped.sum().good
    kstable=kstable.reset_index()
    kstable['bad_rate']=kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks']=(kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value=kstable.ks.abs().max()
    return ks_value

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Función para calcular las métricas de rendimiento
def calcular_metricas(y_verdadero, y_pred, probas, clases):
    print("Informe de clasificación:")
    print(classification_report(y_verdadero, y_pred))

    cm = confusion_matrix(y_verdadero, y_pred)
    cm_df = pd.DataFrame(cm,
                         index=[i for i in clases],
                         columns=[i for i in clases])
    print("Matriz de confusión:")
    print(cm_df)

    print("Precisión del modelo:")
    print(accuracy_score(y_verdadero, y_pred))

    lb = LabelBinarizer()
    lb.fit(y_verdadero)
    y_verdadero_lb = lb.transform(y_verdadero)
    y_pred_lb = lb.transform(y_pred)
    print("Puntuación AUC-ROC para cada clase:")
    for i in range(len(lb.classes_)):
        print(f"{lb.classes_[i]}: {roc_auc_score(y_verdadero_lb[:, i], y_pred_lb[:, i])}")

    print("Valor KS y coeficiente Gini para cada clase:")
    for i in range(len(lb.classes_)):
        data = pd.DataFrame({'label': y_verdadero_lb[:, i], 'probability': probas[:, i]})
        ks = calc_ks(data)
        gini = calc_gini(y_verdadero_lb[:, i], probas[:, i])
        print(f"{lb.classes_[i]}: KS = {ks}, Gini = {gini}")

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas_test = busqueda_en_grilla.predict_proba(X_test)

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
y_pred_test = np.zeros_like(probas_test)
for i in range(3):
    y_pred_test[:, i] = (probas_test[:, i] > thresholds[i]).astype(int)

y_pred_test = np.argmax(y_pred_test, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular las métricas de rendimiento en el conjunto de prueba
print("Métricas de rendimiento en el conjunto de prueba:")
calcular_metricas(y_test, y_pred_test, probas_test, le.classes_)

# Obtener las probabilidades de las predicciones en el conjunto de validación
probas_val = busqueda_en_grilla.predict_proba(X_val)

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
y_pred_val = np.zeros_like(probas_val)
for i in range(3):
    y_pred_val[:, i] = (probas_val[:, i] > thresholds[i]).astype(int)

y_pred_val = np.argmax(y_pred_val, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular las métricas de rendimiento en el conjunto de validación
print("Métricas de rendimiento en el conjunto de validación:")
calcular_metricas(y_val, y_pred_val, probas_val, le.classes_)



In [ ]:
### correc

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Asumiendo que 'X' es tu conjunto de características y 'y' es tu variable objetivo
le = LabelEncoder()
y = le.fit_transform(y)

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Luego, dividir el conjunto temporal en conjuntos de prueba y validación (ambos del 20% del conjunto original)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Crear una instancia del clasificador Gradient Boosting
modelo = GradientBoostingClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda en grilla
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
}

# Realizar la búsqueda en grilla de hiperparámetros
busqueda_en_grilla = GridSearchCV(estimator=modelo,
                                  param_grid=param_grid,
                                  cv=3,
                                  verbose=2,
                                  n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_en_grilla.fit(X_train_res, y_train_res)

# Definir tus umbrales personalizados aquí. Por ejemplo:
umbrales = [0.4, 0.3, 0.3]  # Un umbral para cada clase

# Función para calcular el valor KS
def calc_ks(data):
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    data['bucket']=(data['probability'].rank(pct=True)*10).astype(int)
    grouped=data.groupby('bucket',as_index=True)
    kstable=grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob']=grouped.max().probability
    kstable['bads']=grouped.sum().bad
    kstable['goods']=grouped.sum().good
    kstable=kstable.reset_index()
    kstable['bad_rate']=kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks']=(kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value=kstable.ks.abs().max()
    return ks_value

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Función para calcular las métricas de rendimiento
def calcular_metricas(y_verdadero, y_pred, probas, clases):
    print("Informe de clasificación:")
    print(classification_report(y_verdadero, y_pred))

    cm = confusion_matrix(y_verdadero, y_pred)
    cm_df = pd.DataFrame(cm,
                         index=[i for i in clases],
                         columns=[i for i in clases])
    print("Matriz de confusión:")
    print(cm_df)

    print("Precisión del modelo:")
    print(accuracy_score(y_verdadero, y_pred))

    lb = LabelBinarizer()
    lb.fit(y_verdadero)
    y_verdadero_lb = lb.transform(y_verdadero)
    y_pred_lb = lb.transform(y_pred)
    print("Puntuación AUC-ROC para cada clase:")
    for i in range(len(lb.classes_)):
        print(f"{lb.classes_[i]}: {roc_auc_score(y_verdadero_lb[:, i], y_pred_lb[:, i])}")

    print("Valor KS y coeficiente Gini para cada clase:")
    for i in range(len(lb.classes_)):
        data = pd.DataFrame({'label': y_verdadero_lb[:, i], 'probability': probas[:, i]})
        ks = calc_ks(data)
        gini = calc_gini(y_verdadero_lb[:, i], probas[:, i])
        print(f"{lb.classes_[i]}: KS = {ks}, Gini = {gini}")

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas_test = busqueda_en_grilla.predict_proba(X_test)

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
y_pred_test = np.zeros_like(probas_test)
for i in range(3):
    y_pred_test[:, i] = (probas_test[:, i] > umbrales[i]).astype(int)

y_pred_test = np.argmax(y_pred_test, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular las métricas de rendimiento en el conjunto de prueba
print("Métricas de rendimiento en el conjunto de prueba:")
calcular_metricas(y_test, y_pred_test, probas_test, le.classes_)


In [ ]:
# Obtener las probabilidades de las predicciones en el conjunto de validación
probas_val = busqueda_en_grilla.predict_proba(X_val)

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
y_pred_val = np.zeros_like(probas_val)
for i in range(3):
    y_pred_val[:, i] = (probas_val[:, i] > umbrales[i]).astype(int)

y_pred_val = np.argmax(y_pred_val, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular las métricas de rendimiento en el conjunto de validación
print("Métricas de rendimiento en el conjunto de validación:")
calcular_metricas(y_val, y_pred_val, probas_val, le.classes_)

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas_test = busqueda_en_grilla.predict_proba(X_test)

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
y_pred_test = np.zeros_like(probas_test)
for i in range(3):
    y_pred_test[:, i] = (probas_test[:, i] > umbrales[i]).astype(int)

y_pred_test = np.argmax(y_pred_test, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular las métricas de rendimiento en el conjunto de prueba
print("Métricas de rendimiento en el conjunto de prueba:")
calcular_metricas(y_test, y_pred_test, probas_test, le.classes_)


In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Asumiendo que 'X' es tu conjunto de características y 'y' es tu variable objetivo
le = LabelEncoder()
y = le.fit_transform(y)

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Luego, dividir el conjunto temporal en conjuntos de prueba y validación (ambos del 20% del conjunto original)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Crear una instancia del clasificador Random Forest
modelo = RandomForestClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda en grilla
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
}

# Realizar la búsqueda en grilla de hiperparámetros
busqueda_en_grilla = GridSearchCV(estimator=modelo,
                                  param_grid=param_grid,
                                  cv=3,
                                  verbose=2,
                                  n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_en_grilla.fit(X_train_res, y_train_res)

# Definir tus umbrales personalizados aquí. Por ejemplo:
umbrales = [0.4, 0.3, 0.3]  # Un umbral para cada clase

# Función para calcular el valor KS
def calc_ks(data):
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    data['bucket']=(data['probability'].rank(pct=True)*10).astype(int)
    grouped=data.groupby('bucket',as_index=True)
    kstable=grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob']=grouped.max().probability
    kstable['bads']=grouped.sum().bad
    kstable['goods']=grouped.sum().good
    kstable=kstable.reset_index()
    kstable['bad_rate']=kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks']=(kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value=kstable.ks.abs().max()
    return ks_value

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Función para calcular las métricas de rendimiento
def calcular_metricas(y_verdadero, y_pred, probas, clases):
    print("Informe de clasificación:")
    print(classification_report(y_verdadero, y_pred))

    cm = confusion_matrix(y_verdadero, y_pred)
    cm_df = pd.DataFrame(cm,
                         index=[i for i in clases],
                         columns=[i for i in clases])
    print("Matriz de confusión:")
    print(cm_df)

    print("Precisión del modelo:")
    print(accuracy_score(y_verdadero, y_pred))

    lb = LabelBinarizer()
    lb.fit(y_verdadero)
    y_verdadero_lb = lb.transform(y_verdadero)
    y_pred_lb = lb.transform(y_pred)
    print("Puntuación AUC-ROC para cada clase:")
    for i in range(len(lb.classes_)):
        print(f"{lb.classes_[i]}: {roc_auc_score(y_verdadero_lb[:, i], y_pred_lb[:, i])}")

    print("Valor KS y coeficiente Gini para cada clase:")
    for i in range(len(lb.classes_)):
        data = pd.DataFrame({'label': y_verdadero_lb[:, i], 'probability': probas[:, i]})
        ks = calc_ks(data)
        gini = calc_gini(y_verdadero_lb[:, i], probas[:, i])
        print(f"{lb.classes_[i]}: KS = {ks}, Gini = {gini}")

# Obtener las probabilidades de las predicciones en el conjunto de validación
probas_val = busqueda_en_grilla.predict_proba(X_val)

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
y_pred_val = np.zeros_like(probas_val)
for i in range(3):
    y_pred_val[:, i] = (probas_val[:, i] > umbrales[i]).astype(int)

y_pred_val = np.argmax(y_pred_val, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular las métricas de rendimiento en el conjunto de validación
print("Métricas de rendimiento en el conjunto de validación:")
calcular_metricas(y_val, y_pred_val, probas_val, le.classes_)

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas_test = busqueda_en_grilla.predict_proba(X_test)

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
y_pred_test = np.zeros_like(probas_test)
for i in range(3):
    y_pred_test[:, i] = (probas_test[:, i] > umbrales[i]).astype(int)

y_pred_test = np.argmax(y_pred_test, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular las métricas de rendimiento en el conjunto de prueba
print("Métricas de rendimiento en el conjunto de prueba:")
calcular_metricas(y_test, y_pred_test, probas_test, le.classes_)


In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Asumiendo que 'X' es tu conjunto de características y 'y' es tu variable objetivo
le = LabelEncoder()
y = le.fit_transform(y)

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Luego, dividir el conjunto temporal en conjuntos de prueba y validación (ambos del 20% del conjunto original)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Crear una instancia del clasificador LightGBM
modelo = LGBMClassifier(random_state=42)

# Definir la grilla de hiperparámetros para la búsqueda en grilla
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
}

# Realizar la búsqueda en grilla de hiperparámetros
busqueda_en_grilla = GridSearchCV(estimator=modelo,
                                  param_grid=param_grid,
                                  cv=3,
                                  verbose=2,
                                  n_jobs=-1)

# Entrenar el modelo con los mejores hiperparámetros encontrados
busqueda_en_grilla.fit(X_train_res, y_train_res)

# Definir tus umbrales personalizados aquí. Por ejemplo:
umbrales = [0.4, 0.3, 0.3]  # Un umbral para cada clase

# Función para calcular el valor KS
def calc_ks(data):
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    data['bucket']=(data['probability'].rank(pct=True)*10).astype(int)
    grouped=data.groupby('bucket',as_index=True)
    kstable=grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob']=grouped.max().probability
    kstable['bads']=grouped.sum().bad
    kstable['goods']=grouped.sum().good
    kstable=kstable.reset_index()
    kstable['bad_rate']=kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks']=(kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value=kstable.ks.abs().max()
    return ks_value

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Función para calcular las métricas de rendimiento
def calcular_metricas(y_verdadero, y_pred, probas, clases):
    print("Informe de clasificación:")
    print(classification_report(y_verdadero, y_pred))

    cm = confusion_matrix(y_verdadero, y_pred)
    cm_df = pd.DataFrame(cm,
                         index=[i for i in clases],
                         columns=[i for i in clases])
    print("Matriz de confusión:")
    print(cm_df)

    print("Precisión del modelo:")
    print(accuracy_score(y_verdadero, y_pred))

    lb = LabelBinarizer()
    lb.fit(y_verdadero)
    y_verdadero_lb = lb.transform(y_verdadero)
    y_pred_lb = lb.transform(y_pred)
    print("Puntuación AUC-ROC para cada clase:")
    for i in range(len(lb.classes_)):
        print(f"{lb.classes_[i]}: {roc_auc_score(y_verdadero_lb[:, i], y_pred_lb[:, i])}")

    print("Valor KS y coeficiente Gini para cada clase:")
    for i in range(len(lb.classes_)):
        data = pd.DataFrame({'label': y_verdadero_lb[:, i], 'probability': probas[:, i]})
        ks = calc_ks(data)
        gini = calc_gini(y_verdadero_lb[:, i], probas[:, i])
        print(f"{lb.classes_[i]}: KS = {ks}, Gini = {gini}")

# Obtener las probabilidades de las predicciones en el conjunto de validación
probas_val = busqueda_en_grilla.predict_proba(X_val)

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
y_pred_val = np.zeros_like(probas_val)
for i in range(3):
    y_pred_val[:, i] = (probas_val[:, i] > umbrales[i]).astype(int)

y_pred_val = np.argmax(y_pred_val, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular las métricas de rendimiento en el conjunto de validación
print("Métricas de rendimiento en el conjunto de validación:")
calcular_metricas(y_val, y_pred_val, probas_val, le.classes_)

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas_test = busqueda_en_grilla.predict_proba(X_test)

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
y_pred_test = np.zeros_like(probas_test)
for i in range(3):
    y_pred_test[:, i] = (probas_test[:, i] > umbrales[i]).astype(int)

y_pred_test = np.argmax(y_pred_test, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular las métricas de rendimiento en el conjunto de prueba
print("Métricas de rendimiento en el conjunto de prueba:")
calcular_metricas(y_test, y_pred_test, probas_test, le.classes_)


In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Asumiendo que 'X' es tu conjunto de características y 'y' es tu variable objetivo
le = LabelEncoder()
y = le.fit_transform(y)

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Luego, dividir el conjunto temporal en conjuntos de prueba y validación (ambos del 20% del conjunto original)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Crear una instancia del clasificador Naive Bayes Multinomial
modelo = MultinomialNB()

# Entrenar el modelo
modelo.fit(X_train_res, y_train_res)

# Definir tus umbrales personalizados aquí. Por ejemplo:
umbrales = [0.4, 0.3, 0.3]  # Un umbral para cada clase

# Función para calcular el valor KS
def calc_ks(data):
    data['good']=(data['label']==0).astype(int)
    data['bad']=(data['label']==1).astype(int)
    data['bucket']=(data['probability'].rank(pct=True)*10).astype(int)
    grouped=data.groupby('bucket',as_index=True)
    kstable=grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob']=grouped.max().probability
    kstable['bads']=grouped.sum().bad
    kstable['goods']=grouped.sum().good
    kstable=kstable.reset_index()
    kstable['bad_rate']=kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks']=(kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value=kstable.ks.abs().max()
    return ks_value

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Función para calcular las métricas de rendimiento
def calcular_metricas(y_verdadero, y_pred, probas, clases):
    print("Informe de clasificación:")
    print(classification_report(y_verdadero, y_pred))

    cm = confusion_matrix(y_verdadero, y_pred)
    cm_df = pd.DataFrame(cm,
                         index=[i for i in clases],
                         columns=[i for i in clases])
    print("Matriz de confusión:")
    print(cm_df)

    print("Precisión del modelo:")
    print(accuracy_score(y_verdadero, y_pred))

    lb = LabelBinarizer()
    lb.fit(y_verdadero)
    y_verdadero_lb = lb.transform(y_verdadero)
    y_pred_lb = lb.transform(y_pred)
    print("Puntuación AUC-ROC para cada clase:")
    for i in range(len(lb.classes_)):
        print(f"{lb.classes_[i]}: {roc_auc_score(y_verdadero_lb[:, i], y_pred_lb[:, i])}")

    print("Valor KS y coeficiente Gini para cada clase:")
    for i in range(len(lb.classes_)):
        data = pd.DataFrame({'label': y_verdadero_lb[:, i], 'probability': probas[:, i]})
        ks = calc_ks(data)
        gini = calc_gini(y_verdadero_lb[:, i], probas[:, i])
        print(f"{lb.classes_[i]}: KS = {ks}, Gini = {gini}")

# Obtener las probabilidades de las predicciones en el conjunto de validación
probas_val = modelo.predict_proba(X_val)

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
y_pred_val = np.zeros_like(probas_val)
for i in range(3):
    y_pred_val[:, i] = (probas_val[:, i] > umbrales[i]).astype(int)

y_pred_val = np.argmax(y_pred_val, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular las métricas de rendimiento en el conjunto de validación
print("Métricas de rendimiento en el conjunto de validación:")
calcular_metricas(y_val, y_pred_val, probas_val, le.classes_)

# Obtener las probabilidades de las predicciones en el conjunto de prueba
probas_test = modelo.predict_proba(X_test)

# Aplicar los umbrales a las probabilidades para obtener las predicciones finales
y_pred_test = np.zeros_like(probas_test)
for i in range(3):
    y_pred_test[:, i] = (probas_test[:, i] > umbrales[i]).astype(int)

y_pred_test = np.argmax(y_pred_test, axis=1)  # Convertir las predicciones one-hot a etiquetas

# Calcular las métricas de rendimiento en el conjunto de prueba
print("Métricas de rendimiento en el conjunto de prueba:")
calcular_metricas(y_test, y_pred_test, probas_test, le.classes_)


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import numpy as np

# Supongamos que X es tu matriz de características e y son tus etiquetas

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Crear el clasificador MLP
mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)

# Entrenar el modelo
mlp.fit(X_train, y_train)

# Hacer predicciones en el conjunto de validación y prueba
y_pred_val = mlp.predict(X_val)
y_pred_test = mlp.predict(X_test)

# Calcular la precisión del modelo
print(f'Informe de clasificación para el conjunto de validación:\n{classification_report(y_val, y_pred_val)}')
print(f'Informe de clasificación para el conjunto de prueba:\n{classification_report(y_test, y_pred_test)}')

# Calcular la matriz de confusión
print(f'Matriz de confusión para el conjunto de validación:\n{confusion_matrix(y_val, y_pred_val)}')
print(f'Matriz de confusión para el conjunto de prueba:\n{confusion_matrix(y_test, y_pred_test)}')

# Calcular el valor de AUC-ROC para cada clase
lb = LabelBinarizer()
lb.fit(y)
y_test_lb = lb.transform(y_test)
y_pred_test_lb = lb.transform(y_pred_test)
roc_auc = roc_auc_score(y_test_lb, y_pred_test_lb, average=None)
print(f'Puntuación AUC-ROC para cada clase en el conjunto de prueba: {roc_auc}')

# Calcular el valor de Gini para cada clase
gini = 2*roc_auc - 1
print(f'Coeficiente de Gini para cada clase en el conjunto de prueba: {gini}')

# Calcular el valor de KS para cada clase
for i in range(len(lb.classes_)):
    data = pd.DataFrame({'label': y_test_lb[:, i], 'probability': y_pred_test_lb[:, i]})
    data['good'] = (data['label'] == 0).astype(int)
    data['bad'] = (data['label'] == 1).astype(int)
    data['bucket'] = (data['probability'].rank(pct=True)*10).astype(int)
    grouped = data.groupby('bucket', as_index=True)
    kstable = grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob'] = grouped.max().probability
    kstable['bads'] = grouped.sum().bad
    kstable['goods'] = grouped.sum().good
    kstable = kstable.reset_index()
    kstable['bad_rate'] = kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks'] = (kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value = kstable.ks.abs().max()
    print(f'Valor KS para la clase {lb.classes_[i]} en el conjunto de prueba: {ks_value}')


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_selection import SelectKBest, f_classif
import pandas as pd
import numpy as np

# Supongamos que X es tu matriz de características e y son tus etiquetas

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Crear el selector de características
selector = SelectKBest(f_classif, k=10)  # Ajusta 'k' al número de características que deseas mantener

# Ajustar y transformar los datos de entrenamiento
X_train_new = selector.fit_transform(X_train, y_train)

# Transformar los datos de validación y prueba
X_val_new = selector.transform(X_val)
X_test_new = selector.transform(X_test)

# Crear el clasificador MLP con regularización L2
mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000, alpha=0.01)

# Entrenar el modelo con los nuevos datos
mlp.fit(X_train_new, y_train)

# Hacer predicciones en el conjunto de validación y prueba
y_pred_val = mlp.predict(X_val_new)
y_pred_test = mlp.predict(X_test_new)

# Calcular la precisión del modelo
print(f'Informe de clasificación para el conjunto de validación:\n{classification_report(y_val, y_pred_val)}')
print(f'Informe de clasificación para el conjunto de prueba:\n{classification_report(y_test, y_pred_test)}')

# Calcular la matriz de confusión
print(f'Matriz de confusión para el conjunto de validación:\n{confusion_matrix(y_val, y_pred_val)}')
print(f'Matriz de confusión para el conjunto de prueba:\n{confusion_matrix(y_test, y_pred_test)}')

# Calcular el valor de AUC-ROC para cada clase
lb = LabelBinarizer()
lb.fit(y)
y_test_lb = lb.transform(y_test)
y_pred_test_lb = lb.transform(y_pred_test)
roc_auc = roc_auc_score(y_test_lb, y_pred_test_lb, average=None)
print(f'Puntuación AUC-ROC para cada clase en el conjunto de prueba: {roc_auc}')

# Calcular el valor de Gini para cada clase
gini = 2*roc_auc - 1
print(f'Coeficiente de Gini para cada clase en el conjunto de prueba: {gini}')

# Calcular el valor de KS para cada clase
for i in range(len(lb.classes_)):
    data = pd.DataFrame({'label': y_test_lb[:, i], 'probability': y_pred_test_lb[:, i]})
    data['good'] = (data['label'] == 0).astype(int)
    data['bad'] = (data['label'] == 1).astype(int)
    data['bucket'] = (data['probability'].rank(pct=True)*10).astype(int)
    grouped = data.groupby('bucket', as_index=True)
    kstable = grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob'] = grouped.max().probability
    kstable['bads'] = grouped.sum().bad
    kstable['goods'] = grouped.sum().good
    kstable = kstable.reset_index()
    kstable['bad_rate'] = kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks'] = (kstable.bads/kstable.bads.sum()).cumsum()-(kstable.goods/kstable.goods.sum()).cumsum()
    ks_value = kstable.ks.abs().max()
    print(f'Valor KS para la clase {lb.classes_[i]} en el conjunto de prueba: {ks_value}')


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, auc
from scipy.stats import ks_2samp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Carga tus datos
# df = pd.read_csv('ruta/a/tu/dataset.csv')

# Supongamos que 'X' son tus características y 'y' es tu variable objetivo
# X = df.drop('columna_objetivo', axis=1)
# y = df['columna_objetivo']

# Estandariza los datos de entrada
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Codifica las etiquetas de clase como vectores one-hot
label_binarizer = LabelBinarizer()
y = label_binarizer.fit_transform(y)

# Divide los datos en conjuntos de entrenamiento, prueba y validación
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Define la arquitectura de la DNN
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 3 clases

# Compila el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrena el modelo
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Predice las probabilidades de las clases para los conjuntos de prueba y validación
y_test_probs = model.predict(X_test)
y_val_probs = model.predict(X_val)

# Función para calcular el coeficiente Gini
def calc_gini(y_verdadero, y_prob):
    fpr, tpr, _ = roc_curve(y_verdadero, y_prob)
    roc_auc = auc(fpr, tpr)
    gini = 2*roc_auc - 1
    return gini

# Función para calcular el valor KS
def calc_ks(data):
    data['bucket'] = (data['probability'].rank(pct=True)*10).astype(int)
    grouped = data.groupby('bucket', as_index=True)
    kstable = grouped.min().probability.to_frame(name='min_prob')
    kstable['max_prob'] = grouped.max().probability
    kstable['bads'] = grouped.sum().bad
    kstable['goods'] = grouped.sum().good
    kstable = kstable.reset_index()
    kstable['bad_rate'] = kstable.bads/(kstable.bads+kstable.goods)
    kstable['ks'] = (kstable.bads/kstable.bads.sum()).cumsum() - (kstable.goods/kstable.goods.sum()).cumsum()
    ks_value = kstable.ks.abs().max()
    return ks_value

# Calcular e imprimir el valor KS, el coeficiente Gini y la métrica ROC AUC para cada clase
for i in range(y.shape[1]):
    print(f"\nClase {i+1}:")

    data_test = pd.DataFrame({'probability': y_test_probs[:, i], 'label': y_test[:, i]})
    data_test['good'] = (data_test['label'] == 0).astype(int)
    data_test['bad'] = (data_test['label'] == 1).astype(int)

    data_val = pd.DataFrame({'probability': y_val_probs[:, i], 'label': y_val[:, i]})
    data_val['good'] = (data_val['label'] == 0).astype(int)
    data_val['bad'] = (data_val['label'] == 1).astype(int)

    ks_test = calc_ks(data_test)
    ks_val = calc_ks(data_val)

    gini_test = calc_gini(y_test[:, i], y_test_probs[:, i])
    gini_val = calc_gini(y_val[:, i], y_val_probs[:, i])

    roc_auc_test_ovr = roc_auc_score(y_test[:, i], y_test_probs[:, i])
    roc_auc_val_ovr = roc_auc_score(y_val[:, i], y_val_probs[:, i])

    print(f'KS Test: {ks_test}')
    print(f'KS Validation: {ks_val}')
    print(f'Gini Test: {gini_test}')
    print(f'Gini Validation: {gini_val}')
    print(f'ROC AUC Test (OvR): {roc_auc_test_ovr}')
    print(f'ROC AUC Validation (OvR): {roc_auc_val_ovr}')




In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import roc_curve, auc, roc_auc_score
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# Codifica las etiquetas de clase como números enteros
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)

# Convierte las etiquetas de clase enteras a one-hot
y = np_utils.to_categorical(encoded_Y)

# Define los pesos de las clases manualmente
class_weights = {0: 1.0,  # Peso para la clase 0
                 1: 1.0,  # Peso para la clase 1
                 2: 1.0}  # Peso para la clase 2

# Define el umbral de decisión
threshold = 0.5

# Define la arquitectura de la DNN
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))  # 3 clases

# Compila el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrena el modelo con los pesos de las clases
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32, class_weight=class_weights)

# Predice las probabilidades de las clases para los conjuntos de prueba y validación
y_test_probs = model.predict(X_test)
y_val_probs = model.predict(X_val)

# Convierte las probabilidades a predicciones de clase usando el umbral de decisión
y_test_preds = (y_test_probs >= threshold).astype(int)
y_val_preds = (y_val_probs >= threshold).astype(int)

# Aquí puedes continuar con el cálculo de tus métricas...

